In [ ]:
# !pip show tensorflow numpy keras
# !pip install pillow
!pip install ..

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import RBF
import tensorflow as tf
from PIL import Image
from tensorflow import keras

from keras import Input, Model
from keras.models import load_model, Sequential
from keras.layers import Dense, Flatten, ReLU

import torch
import torch.nn as nn
import torch.optim as optim

from time import time
from scipy.stats import poisson, gamma, norm
from scipy.optimize import minimize_scalar
import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor

# from sddr import Sddr  # Assuming you have pyssdr installed and configured correctly
import logging
from datetime import datetime
from itertools import product

# import torch
logging.basicConfig(level=logging.INFO)
import os
import sys
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# import the sddr module
from sddr import Sddr

2025-03-05 10:04:50.769611: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-05 10:04:51.044669: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/scc/pa6512/miniconda3/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:

def scale_to_range(data, lower=-1, upper=1):
    return (data - np.min(data)) / (np.max(data) - np.min(data)) * (upper - lower) + lower

def save_with_var_name(var, var_name, var_type, save_path, scenario_index):
    if var_type == 'npy':
        np.save(f"{save_path}/{var_name}_{scenario_index}.npy", var)
    if var_type == 'keras':
        var.save(f"{save_path}/{var_name}_{scenario_index}.keras")
    if var_type == 'jpgs':
        images_path = f"{save_path}/{var_name}_{scenario_index}"
        os.makedirs(images_path, exist_ok=True)
        for idx, img in enumerate(var):
            # Normalize and convert to uint8
            normalized_img = (img * 255 / np.max(img)).astype(np.uint8)
            # Convert the NumPy array to an image
            normalized_img = Image.fromarray(normalized_img).convert("L")
            normalized_img.save(f"{images_path}/{var_name}_{scenario_index}_{idx}.jpg")
    if var_type == 'pth':
        var.save(f"{var_name}_{scenario_index}.pth")
    if var_type == 'df':
        var.to_csv(f"{save_path}/{var_name}_{scenario_index}.csv", index=False)

    logging.info(f"Saved {var_name} to {save_path}/{var_name}_{scenario_index}.npy")
    
def read_with_var_name(var_name, var_type, save_path, scenario_index):
    if var_type == 'npy':
        return np.load(f"{save_path}/{var_name}_{scenario_index}.npy")
    if var_type == 'keras':
        return tf.keras.models.load_model(f"{save_path}/{var_name}_{scenario_index}.keras")


In [ ]:
read_path = "../data_generation/output_linear"
scenario_index = f"n_{1000}_rep_{0}"
# U_k = read_with_var_name('U_k', 'npy', read_path, scenario_index)
output_dimension_dnn = 16
save_path = "./outputs_wo_unstructured_nknots_16_batch_32"
X = read_with_var_name('X', 'npy', read_path, scenario_index)
# linear_effects = read_with_var_name('linear_effects', 'npy', read_path, scenario_index)
# print(X.shape)
Z = read_with_var_name('Z', 'npy', read_path, scenario_index)
nonlinear_effects = read_with_var_name('nonlinear_effects', 'npy', read_path, scenario_index)

In [5]:
num_knots = 16
grid_size = 28

In [6]:
distribution_SSDR = "Normal" # compatible form
formulas = {
'loc': f"~ 1 + X1 + X2 + spline(Z1, bs='bs', df={num_knots+3}) + spline(Z2, bs='bs', df={num_knots+3})",
'scale': '~ 1'
}
degrees_of_freedom = {'loc':num_knots+3, 'scale':num_knots+3}
deep_models_dict = {
        'dnn': {
            'model': 
                # nn.Sequential(
                # nn.Flatten(1, -1),
                # nn.Linear(grid_size*grid_size,output_dimension_dnn),
                # nn.ReLU()
                
                nn.Sequential(
                nn.Flatten(1, -1),
                nn.Linear(grid_size*grid_size, 32, bias=False),
                nn.ReLU(),
                nn.Linear(32, 16),
                nn.ReLU(),
                # nn.Linear(16, 1)
                ),
            
            'output_shape': output_dimension_dnn},
    }
train_parameters = {
        'batch_size': 32,              # Smaller batch size due to limited sample size.
        'epochs': 100,
        # 'degrees_of_freedom': {'rate': 3},  # For Poisson; adjust accordingly for other distributions.
        'optimizer': optim.Adam,
        'val_split': 0.15,             # Possibly a higher split for very small samples.
        'early_stop_epsilon': 0.001,
        # 'dropout_rate': 0.01           # Start with 0.01; consider 0.01-0.05 range.
    }

train_parameters['degrees_of_freedom'] = degrees_of_freedom
unstructured_data = {
    'Image' : {
        'path' : f"{read_path}/images_jpg_{scenario_index}/",
        'datatype' : 'image'
    }
    }
ssdr = Sddr(output_dir=save_path,
            distribution=distribution_SSDR,
            formulas=formulas,
            deep_models_dict=deep_models_dict,
            train_parameters=train_parameters,
            modify=True,
            ortho_manual = True,
            use_spline_for_struct = False,
            n_knots = num_knots
            )

# print(Z.shape)
# Step 1: Transpose X and Z
X_transposed = X.T  # Shape (10, 2)
Z_transposed = Z.T  # Shape (10, 2)

# Step 2: Combine X and Z
combined_data = np.hstack((X_transposed, Z_transposed))  # Shape (10, 4)
df = pd.DataFrame(combined_data, columns=['X1', 'X2', 'Z1', 'Z2'])
# df['Image'] = [f'images_jpg_{scenario_index}_{i}.jpg' for i in range(len(df))]
# os.makedirs(save_path, exist_ok=True)
scenario_index = scenario_index + f"_dist_gaussian_homo_SNR_1"

# a = read_with_var_name('a', 'npy', save_path, scenario_index)
# etas = read_with_var_name('etas', 'npy', save_path, scenario_index)
y = read_with_var_name('y', 'npy', read_path, scenario_index)
df['Y'] = y
ssdr.load('./outputs_wo_unstructured_nknots_16_batch_32/ssdr_n_1000_rep_0_dist_gaussian_homo_SNR_1_point_estimates.pth',
          df)
ssdr.train(target="Y", structured_data=df, resume=True)


Using device: cpu
19


/home/scc/pa6512/miniconda3/lib/python3.12/site-packages/sddr/utils/utils.py:395: UserWarning: df too large: Degrees of freedom (df = 19) cannot be larger than the rank of the design matrix (rank = 18). 
            Unpenalized base-learner with df = 18 will be used. Re-consider model specification.
  df_lam = df2lambda(dm_spline, P_val, df_val)


ValueError: could not broadcast input array from shape (0,) into shape (18,18)

In [1]:
ssdr

NameError: name 'ssdr' is not defined

In [6]:
net_loc = ssdr.net.single_parameter_sddr_list['loc']

In [17]:
spline_info = ssdr.prepare_data.dm_info_dict['loc']['spline_info']
non_spline_info = ssdr.prepare_data.dm_info_dict['loc']['non_spline_info']

In [18]:
non_spline_info

{'list_of_non_spline_slices': [slice(0, 1, None),
  slice(1, 2, None),
  slice(2, 3, None)],
 'list_of_non_spline_input_features': [[], ['X1'], ['X2']],
 'list_of_term_names': ['Intercept', 'X1', 'X2']}

In [7]:
ssdr.coeff("loc")

{'Intercept': array([0.06083461], dtype=float32),
 'X1': array([-0.27388215], dtype=float32),
 'X2': array([0.06389987], dtype=float32),
 "spline(Z1, bs='bs', df=24)": array([-0.02716911, -0.1502083 ,  0.08915941,  0.00341093, -0.10176762,
        -0.22853501, -0.1698506 , -0.07311463,  0.06400301, -0.01527868,
        -0.16755626, -0.165629  , -0.05728169,  0.11317274,  0.04690243,
         0.01842467,  0.3821651 ,  0.24816796, -0.14492553, -0.10131226,
         0.23916532,  0.10972033,  0.1978991 ,  0.21067359], dtype=float32),
 "spline(Z2, bs='bs', df=24)": array([ 0.242038  ,  0.36188853, -0.25710744, -0.23560256, -0.11818494,
        -0.04522917, -0.02499777, -0.1784563 , -0.42915365, -0.00850329,
        -0.25722575, -0.41706994, -0.12326404,  0.28168833,  0.08507476,
         0.04096813,  0.26483554,  0.0877103 ,  0.10024875, -0.10896218,
        -0.3801032 , -0.08763045,  0.09155629,  0.28837582], dtype=float32)}

In [9]:
len(ssdr.coeff("loc")["spline(Z1, bs='bs', df=24)"])


24

In [8]:
# Create an empty list to store the result rows.
results = []

# Loop over each parameter group in degrees_of_freedom.
for k in degrees_of_freedom.keys():
    # Get the coefficient dictionary for parameter group k.
    # This dictionary is expected to have keys corresponding to feature names.
    
    # Combine the features (if you want to process both kinds together)
    coeff_dict = ssdr.coeff(k)
    
    for feature in coeff_dict.keys():
        # Extract the point estimate for the feature.
        # (Assuming ssdr.coeff(k)[feature] returns a list/array where the first element is the estimate.)                
        # Append a dictionary with the desired columns.
        results.append({
            'scenario_index': scenario_index,  # scenario_index should be defined in your code
            'param_y': k,
            'param_eta': feature,
            'value': coeff_dict[feature]
        })

# Convert the list of dictionaries to a DataFrame.
df_results = pd.DataFrame(results)

In [9]:

df_results

,scenario_index,param_y,param_eta,value
0,n_100_rep_0_dist_gaussian_homo_SNR_1,loc,X1,[-0.016657928]
1,n_100_rep_0_dist_gaussian_homo_SNR_1,loc,X2,[0.20231669]
2,n_100_rep_0_dist_gaussian_homo_SNR_1,loc,"spline(Z1, bs=""bs"")","[-0.33227366, -0.4185705, -0.04214008, 0.60077..."
3,n_100_rep_0_dist_gaussian_homo_SNR_1,loc,"spline(Z2, bs=""bs"")","[0.1305335, 0.08974188, 0.041038435, 0.25026676]"
4,n_100_rep_0_dist_gaussian_homo_SNR_1,scale,Intercept,[0.3417374]


In [6]:
structured_weights = net_loc.structured_head.weight
structured_weights

Parameter containing:
tensor([[-0.0167,  0.2023, -0.3323, -0.4186, -0.0421,  0.6008,  0.1305,  0.0897,
          0.0410,  0.2503]], requires_grad=True)

In [ ]:
deep_head_weights = net_loc.deep_head.weight

In [7]:
deep_head_weights

Parameter containing:
tensor([[ 0.6015,  0.5054,  0.4911, -0.1429, -0.8494,  0.5513,  0.6371,  0.5734,
         -0.1773,  0.4729,  0.4953,  0.5017, -0.2171,  0.5962, -0.2339,  0.4576]],
       requires_grad=True)

In [8]:
structured_head_weights = net_loc.structured_head.weight
structured_head_bias = net_loc.structured_head.bias

In [9]:
ssdr.dataset

In [ ]:


# Append the method to the Sddr class (if not editing the class source directly)
# For example, if your Sddr class is defined in a module, you can add the method as shown above.

# ---------------------------
# Example usage:
# ---------------------------
# Assume `sddr` is your trained Sddr object and `train_dataset` is your full training dataset.
ssdr.net.eval()  # Ensure dropout is off
full_latent_features = ssdr.get_full_latent_features(ssdr.dataset)

# Optionally, you can also extract the deep head weights.
# For example, for parameter "loc":
if "loc" in ssdr.single_parameter_sddr_list:
    deep_head_weights = ssdr.single_parameter_sddr_list["loc"].deep_head.weight.detach().cpu()
    print("Deep head weights for 'loc':", deep_head_weights)

# Now, full_latent_features is a dictionary (e.g., {'loc': tensor([...]), 'scale': tensor([...])})
# You can, for example, build a new model that takes these latent features as input.


In [14]:
len(ssdr.predict(df, unstructured_data=unstructured_data)[2])


2

In [14]:
ssdr.predict(df, unstructured_data=unstructured_data)[1]['loc']

[(array([0.54340494, 0.27836939, 0.42451759, 0.84477613, 0.00471886,
         0.12156912, 0.67074908, 0.82585276, 0.13670659, 0.57509333,
         0.89132195, 0.20920212, 0.18532822, 0.10837689, 0.21969749,
         0.97862378, 0.81168315, 0.17194101, 0.81622475, 0.27407375,
         0.43170418, 0.94002982, 0.81764938, 0.33611195, 0.17541045,
         0.37283205, 0.00568851, 0.25242635, 0.79566251, 0.01525497,
         0.59884338, 0.60380454, 0.10514769, 0.38194344, 0.03647606,
         0.89041156, 0.98092086, 0.05994199, 0.89054594, 0.5769015 ,
         0.74247969, 0.63018394, 0.58184219, 0.02043913, 0.21002658,
         0.54468488, 0.76911517, 0.25069523, 0.28589569, 0.85239509,
         0.97500649, 0.88485329, 0.35950784, 0.59885895, 0.35479561,
         0.34019022, 0.17808099, 0.23769421, 0.04486228, 0.50543143,
         0.37625245, 0.5928054 , 0.62994188, 0.14260031, 0.9338413 ,
         0.94637988, 0.60229666, 0.38776628, 0.363188  , 0.20434528,
         0.27676506, 0.24653588, 0

In [16]:
len(ssdr.predict(df, unstructured_data=unstructured_data)[1]['loc'][0][0])

100

In [23]:
print(U_k)

[[0.13102287 0.         0.15486272 ... 0.         0.         0.16163117]
 [0.         0.19298601 0.         ... 0.3902848  0.         0.27889434]
 [0.         0.         0.         ... 0.22374676 0.00426558 0.50957525]
 ...
 [0.         0.39148313 0.35076192 ... 0.43440923 0.         0.45768496]
 [0.         0.31406093 0.         ... 0.5324676  0.21987693 0.7418438 ]
 [0.         0.         0.         ... 0.33558524 0.         0.18744768]]


In [ ]:
eval_dict = {}
eval_dict[k] = eval_results

In [ ]:
param_to_index = {"rate": 0, "loc": 0, "scale": 1}
coverage_rates = {}
for param, partial_effects in eval_dict.items():
    if len(partial_effects)>0:
        coverage_rates[param] = {}
        # For Gaussian (or gamma) cases, we assume true_nonlinear_effects is a dict with keys matching the parameter names.
        # For Poisson, true_nonlinear_effects is a list.
        true_effects = nonlinear_effects[:,:,param_to_index[param]]
        for idx, effect in enumerate(partial_effects):
            if len(effect) == 6:
                feature, _, ci950, ci951, _, _ = effect
                
                # Get the corresponding true effect for this spline; shape: (n_samples, )
                true_effect = true_effects[idx, :]
                # plot_true_and_ci(true_effect, effect, param, idx, f"{save_path}/plot_{scenario_index}")
                # Sort both the feature and true effect to ensure proper alignment.
                sort_idx = np.argsort(feature)
                sorted_feature = np.array(feature)[sort_idx]
                sorted_ci950 = ci950[sort_idx]
                sorted_ci951 = ci951[sort_idx]
                sorted_true_effect = true_effect[sort_idx]
                
                # Now compute coverage.
                covered = np.logical_and(sorted_true_effect >= sorted_ci950, sorted_true_effect <= sorted_ci951)
                coverage_rate = np.mean(covered)
                coverage_rates[param][f'spline_{idx}'] = coverage_rate
            else:
                coverage_rates[param][f'spline_{idx}'] = None

In [ ]:
eval_dict = {}
for k in degrees_of_freedom.keys():
    eval_results = ssdr.eval(k, plot=False)
    eval_dict[k] = eval_results

In [ ]:
len(eval_dict['loc'])

In [ ]:
len(eval_dict['loc'][0])

In [ ]:
ssdr.eval('scale', plot=False)

In [ ]:

scenario_index = f"n_{100}_rep_{1}"
nonlinear_effects = read_with_var_name('nonlinear_effects', 'npy', read_path, scenario_index)


In [ ]:
true_nonlinear_effects = nonlinear_effects

In [ ]:
eval_dict

In [ ]:
# Compute coverage rates for each parameter.
coverage_rates = {}
for param, partial_effects in eval_dict.items():
    coverage_rates[param] = {}
    # For Gaussian (or gamma) cases, we assume true_nonlinear_effects is a dict with keys matching the parameter names.
    # For Poisson, true_nonlinear_effects is a list.
    if isinstance(true_nonlinear_effects, dict):
        true_effects = true_nonlinear_effects[param]
    else:
        true_effects = true_nonlinear_effects

    for idx, effect in enumerate(partial_effects):
        if len(effect) == 6:
            _, _, ci950, ci951, _, _ = effect
            true_effect = true_effects[idx]
            covered = np.logical_and(true_effect >= ci950, true_effect <= ci951)
            coverage_rate = np.mean(covered)
            coverage_rates[param][f'spline_{idx}'] = coverage_rate
        else:
            coverage_rates[param][f'spline_{idx}'] = None


In [ ]:
coverage_rates

In [ ]:
# confirm that the penalty matrix is structured correctly: 
# all zeros for the linear components and non-zero values for the spline components.
import torch

# Assume P_rate is the penalty matrix and term_slices is the OrderedDict from design_info:
P_rate = result.prepare_data.P['rate']  # e.g. a torch tensor of shape [11, 11]
term_slices = result.prepare_data.structured_matrix_design_info['rate'].term_name_slices

# Define the term names for linear and spline parts
linear_terms = ['Intercept', 'X1', 'X2']
spline_terms = ['spline(Z1, bs="bs")', 'spline(Z2, bs="bs")']

# Check linear part: should be all zeros.
for term in linear_terms:
    sl = term_slices[term]
    block = P_rate[sl, sl]
    print(f"Penalty block for {term}:")
    print(block)
    is_zero = torch.allclose(block, torch.zeros_like(block))
    print(f"All zeros: {is_zero}\n")

# Check spline part: should be non-zero.
for term in spline_terms:
    sl = term_slices[term]
    block = P_rate[sl, sl]
    print(f"Penalty block for {term}:")
    print(block)
    # Check if there's at least one non-zero element:
    non_zero = not torch.allclose(block, torch.zeros_like(block))
    print(f"Non-zero: {non_zero}\n")


In [ ]:

from patsy import build_design_matrices
spline_info = result.prepare_data.dm_info_dict['rate']['spline_info']
non_spline_info = result.prepare_data.dm_info_dict['rate']['non_spline_info']
structured_matrix = build_design_matrices([result.prepare_data.structured_matrix_design_info['rate']],
                                          data, NA_action='raise', return_type='dataframe')[0]



In [ ]:

for spline_slice, spline_input_features in zip(spline_info['list_of_spline_slices'], 
                                                   spline_info['list_of_spline_input_features']):
    X = structured_matrix.iloc[:,spline_slice]
    constraints = []
    for non_spline_slice, non_spline_input_features in zip(
            non_spline_info['list_of_non_spline_slices'], 
            non_spline_info['list_of_non_spline_input_features']):
        print("non_spline_input_features", set(non_spline_input_features))
        print("spline_input_features", set(spline_input_features))
        if set(non_spline_input_features).issubset(set(spline_input_features)):
            constraints.append(structured_matrix.iloc[:,non_spline_slice].values)
    
    if len(constraints)>0:
        constraints = np.concatenate(constraints,axis=1)

In [ ]:
constraints

In [ ]:
result.prepare_data.network_info_dict['rate']['orthogonalization_pattern']

In [ ]:
result.net.single_parameter_sddr_list['rate']

In [ ]:
result.train_loader

In [ ]:
for batch in result.train_loader:
    # for each batch
    target = batch['target'].float()
    datadict = batch['datadict']

In [ ]:
X = datadict['rate']["structured"]

In [ ]:
datadict['rate'].keys()

In [ ]:
key = 'dnn'

In [ ]:
datadict['rate'][key]

In [ ]:
net = result.prepare_data.deep_models_dict[key]
# Uhat_net = net(datadict['rate'][key])

# orthogonalize the output of the neural network with respect to the parts of the structured part,
# that contain the same input as the neural network
if len(result.prepare_data.network_info_dict['rate']['orthogonalization_pattern'][key]) >0:
    X_sliced_with_orthogonalization_pattern = torch.cat([X[:,sl] for sl in result.prepare_data.network_info_dict['rate']['orthogonalization_pattern'][key]],1)
    Q, R = torch.qr(X_sliced_with_orthogonalization_pattern)
    # Utilde_net = result.prepare_data.network_info_dict['rate']._orthog_layer(Q, Uhat_net)
# else:
    # Utilde_net = Uhat_net

In [ ]:
X_sliced_with_orthogonalization_pattern

In [ ]:
result.prepare_data.network_info_dict['rate']

In [ ]:
result.prepare_data.network_info_dict['rate']['orthogonalization_pattern']

In [ ]:
net

In [ ]:
result.coeff('rate')

In [ ]:
result.get_distribution().mean

In [ ]:

result.get_distribution()

In [ ]:
partial_effects_loc = result.eval('rate',plot=True)

In [ ]:
partial_effects_loc

In [ ]:
len(partial_effects_loc)

In [ ]:
feature, partial_effect, ci950, ci951, ci250, ci251 = partial_effects_loc[0]

In [ ]:
feature

In [ ]:
partial_effect

In [ ]:

ci950

In [ ]:
ci250

In [ ]:
method = ['deep_ensemble', 'dropout_sampling', 'last_layer']
# n_list = [100, 500, 1000]
n_list = [10]
distribution_list = ["poisson", "gamma", "gaussian"]
SNR_list=[1,8]

In [ ]:
combinations = list(product(distribution_list, SNR_list, method))
combinations

In [ ]:
def read_with_var_name(var_name, var_type, save_path, scenario_index):
    if var_type == 'npy':
        return np.load(f"{save_path}/{var_name}_{scenario_index}.npy")
    if var_type == 'keras':
        return tf.keras.models.load_model(f"{save_path}/{var_name}_{scenario_index}.keras")

scenario_index = f"n_{10}_rep_{4}"
save_path = "../data_generation/output_debug_local"
U_k = read_with_var_name('U_k', 'npy', save_path, scenario_index)

In [ ]:
ssdr.dataset.prepared_data['loc'].keys()

In [ ]:
etas = np.zeros((10, 2))

In [ ]:
etas[:, 1] = 1

In [ ]:
etas[:, 1]